# Practice training a deep neural network on the CIFAR10 image dataset:

In [1]:
import os
os.chdir("..")

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
import time

import numpy as np
from pathlib import Path

a. Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the Swish activation function.

In [3]:
batch_size = 64

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
)

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data into train and validation subsets (e.g., 80%/20%)
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [118]:
def weights_init(layer, nonlinearity='leaky_relu'):
    if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity=nonlinearity)
        torch.nn.init.constant_(layer.bias, 0)

class Swish(nn.Module):
    def forward(self, x):
        return x * F.sigmoid(x)

In [5]:
class CIFAR10(nn.Module):
    def __init__(self, input_features=3*32*32, output_neurons=100, num_classes=10, hidden_layers=20):
        super(CIFAR10, self).__init__()
        layers = []
        
        layer_first = nn.Linear(input_features, output_neurons)
        layers.append(layer_first)
        layers.append(Swish())
        
        for i in range(hidden_layers-1):
            layer = nn.Linear(output_neurons, output_neurons)
            layers.append(layer)
            layers.append(Swish())

        layer_last = nn.Linear(output_neurons, num_classes)
        layers.append(layer_last)
        
        self.net = nn.Sequential(*layers)
        self.net.apply(weights_init)

    def forward(self, X):
        flatten = nn.Flatten()
        X = flatten(X)
        return self.net(X)

In [4]:
def train(model, train_loader, val_loader, criterion, optimizer, device, epochs=50, patience=5, standardization=None, standardize=False):
    best_val_loss = float('inf')
    patience_counter = 0
    start = time.time()
    for epoch in range(epochs):
        model.train()
        per_epoch_train_loss = 0.0
        time_per_epoch = time.time()
        for data in train_loader:
            inputs, targets = data[0].to(device), data[1].to(device)
            
            optimizer.zero_grad()

            if standardize:
                inputs = standardization(inputs)
                
            outputs = model(inputs)
            loss = criterion(outputs, targets)            
            loss.backward()
            optimizer.step()
            per_epoch_train_loss += loss.item()

        avg_per_epoch_train_loss = per_epoch_train_loss / len(train_loader)
        writer.add_scalar("Loss/Train", avg_per_epoch_train_loss, epoch+1)

        per_epoch_val_loss = 0.0
        total = 0
        correct = 0
        for data in val_loader:
            inputs, targets = data[0].to(device), data[1].to(device)
            
            with torch.no_grad():
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                loss = criterion(outputs, targets)                
                per_epoch_val_loss += loss.item()
                
        avg_per_epoch_val_loss = per_epoch_val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        writer.add_scalar("Loss/Val", avg_per_epoch_val_loss, epoch+1)
        writer.add_scalar("Accuracy/Val", val_accuracy, epoch+1)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_per_epoch_train_loss:.4f}, " 
              f"Val Loss: {avg_per_epoch_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%, Time Elapsed {time.time() - time_per_epoch:.3f}s")

        if  avg_per_epoch_val_loss < best_val_loss:
            patience_counter = 0
            best_val_loss = avg_per_epoch_val_loss
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                model.load_state_dict(best_model_state)
                break

    end = time.time()
    print(f"\nTotal Time for Training {(end-start)/60:.3f}m")
    return model

In [5]:
def eval(model, test_loader, device='cpu'):
    model.eval()
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        inputs  = inputs.to(device)
        targets = targets.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [364]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter()

cifar10  = CIFAR10()
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(cifar10.parameters(), lr=0.001)
model = train(cifar10, train_loader, val_loader, criterion, optimizer, device=device, epochs=5)
writer.flush()
writer.close()

Epoch [1/5], Train Loss: 1.9001, Val Loss: 1.7899, Val Acc: 33.84%, Time Elapsed 56.610s
Epoch [2/5], Train Loss: 1.6878, Val Loss: 1.6607, Val Acc: 40.21%, Time Elapsed 57.438s
Epoch [3/5], Train Loss: 1.5644, Val Loss: 1.5988, Val Acc: 41.87%, Time Elapsed 50.313s
Epoch [4/5], Train Loss: 1.4805, Val Loss: 1.5559, Val Acc: 45.23%, Time Elapsed 54.452s
Epoch [5/5], Train Loss: 1.4158, Val Loss: 1.5333, Val Acc: 45.58%, Time Elapsed 51.567s

Total Time for Training 4.506m


In [379]:
eval(model, test_loader, device=device)

Test Accuracy: 48.54%


In [8]:
class CIFAR10V2(nn.Module):
    def __init__(self, input_features=3*32*32, output_neurons=100, num_classes=10, hidden_layers=20):
        super(CIFAR10V2, self).__init__()
        layers = []
        
        layer_first = nn.Linear(input_features, output_neurons)
        layers.append(layer_first)
        layers.append(nn.BatchNorm1d(output_neurons))
        layers.append(Swish())
        
        for i in range(hidden_layers-1):
            layer = nn.Linear(output_neurons, output_neurons)
            layers.append(layer)
            layers.append(nn.BatchNorm1d(output_neurons))
            layers.append(Swish())

        layer_last = nn.Linear(output_neurons, num_classes)
        layers.append(layer_last)
        
        self.net = nn.Sequential(*layers)
        self.net.apply(weights_init)

    def forward(self, X):
        flatten = nn.Flatten()
        X = flatten(X)
        return self.net(X)

In [378]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter()

cifar10v2  = CIFAR10V2()
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(cifar10v2.parameters(), lr=0.001)
model = train(cifar10v2, train_loader, val_loader, criterion, optimizer, device=device, epochs=5)
writer.flush()
writer.close()

Epoch [1/5], Train Loss: 2.0241, Val Loss: 1.8095, Val Acc: 33.79%, Time Elapsed 61.484s
Epoch [2/5], Train Loss: 1.7113, Val Loss: 1.6768, Val Acc: 40.24%, Time Elapsed 61.726s
Epoch [3/5], Train Loss: 1.6001, Val Loss: 1.6090, Val Acc: 43.22%, Time Elapsed 60.703s
Epoch [4/5], Train Loss: 1.5270, Val Loss: 1.5700, Val Acc: 44.20%, Time Elapsed 62.025s
Epoch [5/5], Train Loss: 1.4740, Val Loss: 1.5388, Val Acc: 45.88%, Time Elapsed 61.364s

Total Time for Training 5.122m


In [6]:
def weights_init_lecun(layer):
    if isinstance(layer, nn.Linear):
        gain = np.sqrt(sum(layer.weight.shape) / layer.weight.shape[1])
        torch.nn.init.xavier_normal_(layer.weight, gain=gain)
        torch.nn.init.constant_(layer.bias, 0)

In [141]:
class CIFAR10V3(nn.Module):
    def __init__(self, input_features=3*32*32, output_neurons=100, num_classes=10, hidden_layers=20):
        super(CIFAR10V3, self).__init__()
        layers = []
        
        layer_first = nn.Linear(input_features, output_neurons)
        layers.append(layer_first)
        layers.append(nn.SELU())
        
        for i in range(hidden_layers-1):
            layer = nn.Linear(output_neurons, output_neurons)
            layers.append(layer)
            layers.append(nn.SELU())

        layer_last = nn.Linear(output_neurons, num_classes)
        layers.append(layer_last)
        
        self.net = nn.Sequential(*layers)
        self.net.apply(weights_init_lecun)

    def forward(self, X):
        flatten = nn.Flatten()
        X = flatten(X)
        return self.net(X)

In [146]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter()
standardize = transforms.Compose(
     [transforms.Normalize((0, 0, 0), (1, 1, 1))]
)

cifar10v3  = CIFAR10V3()
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(cifar10v3.parameters(), lr=0.001)
model = train(cifar10v3, train_loader, val_loader, criterion, optimizer, device=device, epochs=5)
writer.flush()
writer.close()
eval(model, test_loader, device=device)

Epoch [1/5], Train Loss: 2.1656, Val Loss: 2.0423, Val Acc: 28.54%, Time Elapsed 22.917s
Epoch [2/5], Train Loss: 1.7683, Val Loss: 2.1380, Val Acc: 30.68%, Time Elapsed 23.921s
Epoch [3/5], Train Loss: 1.6571, Val Loss: 1.7635, Val Acc: 39.64%, Time Elapsed 26.930s
Epoch [4/5], Train Loss: 1.5897, Val Loss: 2.0045, Val Acc: 32.44%, Time Elapsed 52.490s
Epoch [5/5], Train Loss: 1.5328, Val Loss: 1.7638, Val Acc: 39.46%, Time Elapsed 52.903s

Total Time for Training 2.988m


In [7]:
class CIFAR10V4(nn.Module):
    def __init__(self, input_features=3*32*32, output_neurons=100, num_classes=10, hidden_layers=20):
        super(CIFAR10V4, self).__init__()
        layers = []
        
        layer_first = nn.Linear(input_features, output_neurons)
        layers.append(layer_first)
        layers.append(nn.SELU())
        
        for i in range(hidden_layers-1):
            layer = nn.Linear(output_neurons, output_neurons)
            layers.append(layer)
            layers.append(nn.SELU())

        layers.append(nn.AlphaDropout(p=0.1))
        layer_last = nn.Linear(output_neurons, num_classes)
        layers.append(layer_last)
        
        self.net = nn.Sequential(*layers)
        self.net.apply(weights_init_lecun)

    def forward(self, X):
        flatten = nn.Flatten()
        X = flatten(X)
        return self.net(X)

In [168]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter()

cifar10v4  = CIFAR10V4()
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(cifar10v4.parameters(), lr=0.001)
model = train(cifar10v4, train_loader, val_loader, criterion, optimizer, device=device, epochs=5)
writer.flush()
writer.close()
eval(model, test_loader, device=device)

Epoch [1/5], Train Loss: 2.1893, Val Loss: 2.3306, Val Acc: 24.92%, Time Elapsed 57.351s
Epoch [2/5], Train Loss: 1.7998, Val Loss: 1.7783, Val Acc: 37.08%, Time Elapsed 53.170s
Epoch [3/5], Train Loss: 1.7017, Val Loss: 1.7400, Val Acc: 40.48%, Time Elapsed 51.750s
Epoch [4/5], Train Loss: 1.6410, Val Loss: 1.8284, Val Acc: 36.38%, Time Elapsed 53.184s
Epoch [5/5], Train Loss: 1.5821, Val Loss: 1.9535, Val Acc: 34.10%, Time Elapsed 54.880s

Total Time for Training 4.508m
Test Accuracy: 34.48%


In [176]:
cifar10v4  = CIFAR10V4()
cifar10v4 = cifar10v4.train(True)

In [196]:
class MCAlphaDropout(nn.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs).train(True)

In [197]:
mc_model = nn.Sequential(*[
    MCAlphaDropout(layer.p)
    if isinstance(layer, nn.AlphaDropout)
    else layer
    for layer in cifar10v4.net
])

In [260]:
t = torch.tensor(t.reshape(-1))

torch.Size([10000])

In [293]:
_, p = torch.max(torch.tensor(y_scores), 1)
p = p.view(-1, 1)

In [200]:
correct = 0
total = targets.size(0)
correct += (predicted == targets).sum().item()
        
print(f"Test Accuracy: {100 * correct / total:.2f}%")

torch.Size([64, 3, 32, 32])

In [207]:
flatten = nn.Flatten()
X = flatten(inp)
mc_model(X[:1])

tensor([[ 4.2214, -9.3684,  7.8547, -6.9430, -3.6141,  1.9189,  0.3642,  4.5611,
         -4.2872,  0.9910]], grad_fn=<AddmmBackward0>)

In [243]:
def mc_dropout_predictions(model, test_loader, device='cpu'):
    model.train(True)
    correct = 0
    total = 0
    y_probas = []
    for inputs, targets in test_loader:
        inputs  = inputs.to(device)
        targets = targets.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            outputs = outputs.detach().numpy()
            y_probas.append(outputs)
            
    return np.vstack(y_probas)

In [249]:
t = np.vstack([targets.view(-1, 1) for _, targets in test_loader])   

In [290]:
y_probas = np.stack([mc_dropout_predictions(cifar10v4, test_loader) for i in range(2)])

In [291]:
y_scores = y_probas.mean(axis=0)

In [119]:
cifar10v4.train(False)
cifar10v4(inp[:1]).round()

tensor([[ 3.,  2., 12., -2., -5., 10., 12., -7., -2., 13.]],
       grad_fn=<RoundBackward0>)

In [8]:
%reload_ext tensorboard
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 20852), started 2 days, 14:03:45 ago. (Use '!kill 20852' to kill it.)